In [1]:
%load_ext autoreload
%autoreload 2

In [60]:
from analyzer import *
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [61]:
labels = ["l1", "l2"]
X, y = make_classification(n_samples=1000, n_features=4, n_informative=2, n_redundant=0, random_state=0, shuffle=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [68]:
analyzer = Analyzer(root_dir="..")

In [70]:
analyzer.model_overview(y_test, y_pred, labels, "1.0", plot_metrics=False, overwrite=False)

/home/sixtenbohman/dev/python/AI_analyzer/venv/lib/python3.11/site-packages/sklearn/metrics/_plot/confusion_matrix.py:136: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()


In [ ]:
analyzer.compare_models("1.0", "1.1", overwrite=False)

In [ ]:
analyzer.to_pdf("1.0")